In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

This is the Managed Cluster, Hub Cluster Info Gathered from thanos, a total of 35 days worth of data.

In [19]:
#cpu_ts = pd.read_csv('/Users/sponnur/PycharmProjects/pythonProject1/cpu_usage_2022_03_25_35_15m.csv')
managed_cluster_df = pd.read_csv('/Users/sponnur/PycharmProjects/pythonProject1/managed_cluster_2022_03_26_35_15m.csv')

In [5]:
managed_cluster_df.head()

,_id,hub_cluster_id,managed_cluster_id,vendor
0,001ce673-3b85-489f-a443-88df28830a56,001ce673-3b85-489f-a443-88df28830a56,001ce673-3b85-489f-a443-88df28830a56,OpenShift
1,01a754f2-8cdb-4f28-b2e1-8fe592d5c1e1,01a754f2-8cdb-4f28-b2e1-8fe592d5c1e1,vanilla-k8s-cluster,Other
2,01a754f2-8cdb-4f28-b2e1-8fe592d5c1e1,01a754f2-8cdb-4f28-b2e1-8fe592d5c1e1,01a754f2-8cdb-4f28-b2e1-8fe592d5c1e1,OpenShift
3,01a754f2-8cdb-4f28-b2e1-8fe592d5c1e1,01a754f2-8cdb-4f28-b2e1-8fe592d5c1e1,d1e5c4e2-590a-4c86-bf24-3c525c824616,OpenShift
4,01ca0384-73e4-4948-a8d2-8f2e3629542b,01ca0384-73e4-4948-a8d2-8f2e3629542b,01ca0384-73e4-4948-a8d2-8f2e3629542b,OpenShift


In [20]:
managed_cluster_df = managed_cluster_df.drop_duplicates(['_id','managed_cluster_id','hub_cluster_id'],keep= 'last')

In [21]:
managed_cluster_df.shape

(7176, 4)

All the _id's in the cluster are Hub Clusters

In [42]:
managed_cluster_df.loc[managed_cluster_df['_id']!=managed_cluster_df['hub_cluster_id']].shape

(0, 5)

In [27]:
managed_cluster_df['managed_count'] = managed_cluster_df.groupby('hub_cluster_id')['hub_cluster_id'].transform('count')

Description of number of managed clusters under the hub cluster's

In [29]:
managed_cluster_df['managed_count'].describe()

count    7176.000000
mean        8.256410
std        16.613895
min         1.000000
25%         2.000000
50%         3.000000
75%         5.000000
max        98.000000
Name: managed_count, dtype: float64

In [25]:
managed_cluster_df.loc[managed_cluster_df['_id'] == '001ce673-3b85-489f-a443-88df28830a56']

,_id,hub_cluster_id,managed_cluster_id,vendor,managed_count
31543,001ce673-3b85-489f-a443-88df28830a56,001ce673-3b85-489f-a443-88df28830a56,001ce673-3b85-489f-a443-88df28830a56,OpenShift,1


All the Managed_cluster_id's in the hub cluster are unique in the dataset (the hub cluster with maximum number of managed cluster)

In [31]:
len(managed_cluster_df.loc[managed_cluster_df['managed_count'] == 98]['managed_cluster_id'].unique())

98

In [11]:
managed_cluster_df.loc[managed_cluster_df['hub_cluster_id'] == managed_cluster_df['managed_cluster_id']].shape

(3144, 4)

In [119]:
actual_managed_clusters = managed_cluster_df.loc[(managed_cluster_df['hub_cluster_id'] != managed_cluster_df['managed_cluster_id']) & (managed_cluster_df['managed_count'] > 1)]['managed_cluster_id'].unique()

In [66]:
all_ids = managed_cluster_df['_id'].unique()

In [67]:
len(all_ids)

3180

Checking if there are any hub cluster's in managed cluster list

In [71]:
correct_ids = []
wrong_ids = []
for cluster_id in actual_managed_clusters:
    if cluster_id in all_ids:
        wrong_ids.append(cluster_id)
    else:
        correct_ids.append(cluster_id)

In [73]:
len(correct_ids), len(wrong_ids), wrong_ids[:5]

(3860,
 39,
 ['1e0735c2-93ed-49fe-88be-c6680e1a5f4f',
  'e2f209a5-c8bf-4217-bce3-a41852d793c9',
  'd47ff408-3698-4c3f-ac6a-8832b846e67c',
  '64049534-9100-4f3c-a7a2-e747e7e10067',
  '17a68f83-34ca-43fc-9c28-8574b008cda8'])

In [101]:
subset_df = managed_cluster_df[managed_cluster_df['_id'].isin(all_ids)][['_id','vendor']]

In [102]:
subset_df = subset_df.drop_duplicates(['_id'],keep= 'last')

In [106]:
subset_df = subset_df.reset_index(drop=True)

In [110]:
subset_df['is_managed'] = 0

In [111]:
subset_df.head()

,_id,vendor,is_managed
0,09b2a0d2-0a08-49b0-9754-76f08b527cdc,OpenShift,0
1,0aadd727-53e2-48a4-b141-6c1f36ed7125,OpenShift,0
2,1ebb492f-971d-4241-b5bb-09c999145293,OpenShift,0
3,3f34fb27-98d8-4264-b4e1-f5303ace1554,OpenShift,0
4,41336a57-f033-4a77-b255-8efac77eafb0,OpenShift,0


In [112]:
subset_df.to_csv('hub_cluster.csv',index=False)

In [113]:
subset_managed_df = pd.DataFrame(correct_ids,columns=['_id'])

In [114]:
subset_managed_df['is_managed'] = 1

In [120]:
subset_managed_df.head()

,_id,is_managed
0,2bc75860-484d-4d42-ae6f-51c24ef8d8c1,1
1,610f1232-d893-4d57-963c-231f770ac263,1
2,16df9f06-c73b-4056-a8af-520c706b8da5,1
3,fe357075-7bfe-46cc-b2d6-23cb6529bd6e,1
4,f6e140ed-ace2-42f4-9f51-ec83ddb59478,1


In [116]:
subset_managed_df.to_csv("managed_clusters.csv",index=False)